In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%pylab
from scipy.optimize import curve_fit

import tkinter as tk
from tkinter import filedialog

import os

from scipy import signal
import scipy.integrate as integrate
import matplotlib.ticker as ticker

# %matplotlib qt

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
%run functions
matplotlib.rcParams.update({'font.size': 25})
cm = pylab.get_cmap('tab10')
cm2 = pylab.get_cmap('Set1')
cm3 = pylab.get_cmap('turbo')

In [3]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.0e6*2*pi # 2.047e6
tauRF = 1/(Omega/2/pi)

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol

wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [204]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

# Use 20220617_extractdatatonpz before

# Data is in the file /home/adrien/RemoteFS/Rivendel/Simulations/20220617

file_path = filedialog.askopenfilename(multiple=True) # initialdir = dir_string
# print(file_path)

time        = []
T_aux_avg   = []
r2_v2_rlim  = []
alpha       = []

for i,j in enumerate(file_path):
    print(j)
    with load(j) as data:
        time.append(data['time'])
        T_aux_avg.append(data['temp'])
        r2_v2_rlim.append(data['r2_v2_rlim'])
        alpha.append(data['alpha'])

/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC05_RF05.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC06_RF06.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC09_RF09.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC10_RF10.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC11_RF11.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC12_RF12.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC13_RF13.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0512_DC03_RF03.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHE

In [378]:
def func2(t, t0, A, B,  C, D):
    return A*(t-t0)/(C+(t-t0)**2)**D + B
def func2abs(t, t0, A, B,  C, D):
    return A*(t-t0)/(C+abs(t-t0))**D + B
def func5(t, t0, A, B, C, k):
    return A*(t-t0) / (C+abs(t-t0)**k)**(1/k) + B
def dfunc5_dt(t,t0,A,B,C,k):
    return A*C*(abs(t-t0)**k+C)**( -(k+1)/k )
#     return A*(C+abs(t-t0)**k)**(-1/k)*(1 - ((t-t0)**2*abs(t-t0)**(k-2))/(C+abs(t-t0)**k) )

def func_lin(t,a,b):
    return a*t+b
def func_exp(x,a,b):
    return b*exp(-a*x)
def func_pow(t,A,B):
    return t**A * 10**(B)
def func_pow2(t,A,B):
    return exp(B)*t**(A)

def omega_ratio_drewsen(alpha):
    alpha_diff = sqrt(1-alpha**(-2))
    return -2* ( arcsin(alpha_diff) - alpha*alpha_diff ) / ( arcsin(alpha_diff) - 1/alpha*alpha_diff )

def omega_ratio_drewsen2(alpha):
    alpha_diff = sqrt(alpha**(-2)-1)
    return -2* ( arcsinh(alpha_diff) - alpha*alpha_diff ) / ( arcsinh(alpha_diff) - 1/alpha*alpha_diff )

In [213]:
linspace(1,3,10)

array([1.        , 1.22222222, 1.44444444, 1.66666667, 1.88888889,
       2.11111111, 2.33333333, 2.55555556, 2.77777778, 3.        ])

/tmp/ipykernel_917878/1122314285.py:20: RuntimeWarning: invalid value encountered in double_scalars
  return -2* ( math.asin(alpha_diff) - alpha*alpha_diff ) / ( math.asin(alpha_diff) - 1/alpha*alpha_diff )


nan

In [216]:
figure('test')
plot(linspace(1.01,3,10),omega_ratio_drewsen(linspace(1.01,3,10)))

ValueError: x, y, and format string must not be None

In [201]:
for i,j in enumerate(file_path):
    print(i,j)

0 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC05_RF05.npz
1 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC06_RF06.npz
2 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz
3 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz
4 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC09_RF09.npz
5 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC10_RF10.npz
6 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC11_RF11.npz
7 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC12_RF12.npz
8 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC13_RF13.npz
9 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0512_DC03_RF03.npz
10 /home/adrien/RemoteFS/Rivendel/Simulations/2022

In [394]:
# Voltages and parameters used in the code

Urf = array([0, 0, 0, 0, 41.065, 55.35111, 57.40115, 59.45119, 61.50124, 63.55128, 65.60132, 67.65136, 69.70140, 71.75144, 0, 0, 0, 0, 0,
             64.6, 62.5, 62.83, 63.03, 63.24, 63.44, 63.86, 64.06, 64.27, 64.47, 62.7262, 62.93, 63.14, 63.34, 63.55, 63.75, 63.96, 64.16, 64.37, 65.74])
q = array([0, 0, 0, 0, 0.4, 0.54, 0.56,  0.58,  0.6, 0.62,  0.64,  0.66,  0.68,  0.7, 0, 0, 0, 0, 0,
           0.63, 0.61, 0.612, 0.614, 0.616, 0.618, 0.622, 0.624, 0.66, 0.68, 0.611, 0.613, 0.615, 0.617, 0.619, 0.621, 0.623, 0.625, 0.627, 0.629])
Udc = array([0, 0, 0, 0, 5.706, 10.41491735, 11.26284989, 12.15184042, 13.08323245, 14.05846827, 15.07909652, 16.14678051, 17.2633073, 18.43059758, 0, 0, 0, 0, 0,
             16.0, 14.8, 14.95, 15.07, 15.19, 15.31, 15.56, 15.68, 15.81, 15.94, 14.89, 15.01, 15.13, 15.25, 15.37, 15.50, 15.62, 15.75, 15.87, 16.00])
N_ions = array([256,256,256,256,256,256,256,256,256,512,512,512,512,512,512,512,512,512,512,512,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,
                1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024])
# K = 1e5*2*pi
# beta_guess = 0.423
# a = 2*K**2/(Omega**2)*Udc
# for k in range(50):
#     beta_guess = beta_continue_alamano(a,q,beta_guess)
#     beta_guess = sqrt(beta_guess)

# omega_z_2 = ( K )**2 * array(Udc)
# omega_x_2 = ( beta_guess*Omega/2 )**2
# omega_r_2 = omega_x_2 - 0.5*omega_z_2

K = 1e5*2*pi
print('q =',q)
beta_guess = 0.423
omega_z_2 = K**2*Udc
az = 2*K**2/Omega**2*Udc
au = -0.5 * az
for i in range(15):
    for k in range(50):
        beta_guess = beta_continue_alamano(au,q,beta_guess)
        beta_guess = sqrt(beta_guess)
omega_r_2 = ( beta_guess*Omega/2 )**2

# display for one given run
to_fit = 2
my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
print(sqrt(omega_r_2[9])/2/pi)
print(sqrt(omega_z_2[9])/2/pi)

x  = time[to_fit]-1e-3
fs = 1/diff(x)[1]

q = [0.    0.    0.    0.    0.4   0.54  0.56  0.58  0.6   0.62  0.64  0.66
 0.68  0.7   0.    0.    0.    0.    0.    0.63  0.61  0.612 0.614 0.616
 0.618 0.622 0.624 0.66  0.68  0.611 0.613 0.615 0.617 0.619 0.621 0.623
 0.625 0.627 0.629]
430716.50838538195
374946.2397464469


In [401]:
print((2*430716.50838538195+3*374946.2397464469)/2e6)
print((5*430716.50838538195-374946.2397464469)/2e6)

0.9931358680050522
0.8893181510902314


In [203]:
# Display R/L aspect ratio
for cond in range(len(file_path)):
    #print(r2_v2_rlim[myRF][k+6][50]*1000) # [condition][dim][time]
    my_try = int(file_path[cond][file_path[cond].find('DC')+2:file_path[cond].find('DC')+4])
    R = mean(r2_v2_rlim[cond][6][100:500])
    L = mean(r2_v2_rlim[cond][8][100:500])
    print(file_path[cond])
    print(q[my_try])
    print('R/L   ',R/L)
    print('alpha ',alpha[cond])
    print('5fx-fz =', (5*sqrt(omega_x_2[my_try])/2/pi-sqrt(omega_z_2[my_try])/2/pi)*1e-6)
    print('')
address = file_path[0]
print(address)

/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC05_RF05.npz
0.54
R/L    1.0898290477202828
alpha  0.5241496731769946
5fx-fz = 2.0778534000679265

/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC06_RF06.npz
0.56
R/L    1.1202091487672696
alpha  0.7102296424097371
5fx-fz = 2.174658382109855

/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz
0.58
R/L    1.1183648010110716
alpha  0.9105842661583182
5fx-fz = 2.2749306358763324

/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz
0.6
R/L    1.14090035852496
alpha  1.1087535032479903
5fx-fz = 2.3792192204599867

/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC09_RF09.npz
0.62
R/L    1.1446405776100723
alpha  1.3963939983344797
5fx-fz = 2.488215166160986

/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC10_RF10.npz
0.64
R/L    1.172

IndexError: list index out of range

In [301]:
omega_ratio_drewsen(r_l)

/tmp/ipykernel_917878/1850721331.py:20: RuntimeWarning: invalid value encountered in true_divide
  return -2* ( arcsin(alpha_diff) - alpha*alpha_diff ) / ( arcsin(alpha_diff) - 1/alpha*alpha_diff )


array([       nan, 1.00240261, 1.00480563, 1.00720905, 1.00961289,
       1.01201713, 1.01442177, 1.01682682, 1.01923226, 1.02163811,
       1.02404435, 1.02645098, 1.02885801, 1.03126543, 1.03367325,
       1.03608145, 1.03849003, 1.040899  , 1.04330836, 1.0457181 ,
       1.04812822, 1.05053871, 1.05294959, 1.05536084, 1.05777247,
       1.06018446, 1.06259683, 1.06500957, 1.06742268, 1.06983615,
       1.07224999, 1.07466419, 1.07707876, 1.07949368, 1.08190897,
       1.08432461, 1.08674061, 1.08915696, 1.09157367, 1.09399073,
       1.09640814, 1.0988259 , 1.10124401, 1.10366246, 1.10608126,
       1.1085004 , 1.11091989, 1.11333971, 1.11575988, 1.11818038,
       1.12060122, 1.12302239, 1.1254439 , 1.12786575, 1.13028792,
       1.13271043, 1.13513326, 1.13755642, 1.13997991, 1.14240372,
       1.14482786, 1.14725231, 1.14967709, 1.15210219, 1.15452761,
       1.15695335, 1.1593794 , 1.16180577, 1.16423245, 1.16665945,
       1.16908675, 1.17151437, 1.17394229, 1.17637052, 1.17879

In [ ]:
# Compute R/L with given wz/wr using Drewsen formula
# omega_ratio_drewsen(alpha)

# generate array with R/L
r_l = linspace(1.01,3,1000

print(ratio_w[7],the_RL)

# The frequency ratio from simu
ratio_w = omega_z_2[4:]/omega_r_2[4:]

# The aspect ratio deduced from Drewsen formula
the_RL = r_l[argmin(abs(ratio_w[7] - omega_ratio_drewsen(r_l)))]

figure('Drewsen eq')
plot()

In [351]:
# Compute R/L with given wz/wr using Drewsen formula
# omega_ratio_drewsen(alpha)

# generate array with R/L

r_l = linspace(1.01,2.5,20)
r_l2 = linspace(0.01,0.999,50)

print(ratio_w[7],the_RL)

# The frequency ratio from simu
ratio_w = omega_z_2[4:]/omega_r_2[4:]

# The aspect ratio deduced from Drewsen formula
the_RL = r_l[argmin(abs(ratio_w[7] - omega_ratio_drewsen(r_l)))]

figure('Drewsen eq',clear='True')
plot(sqrt(omega_ratio_drewsen(r_l)),r_l)
plot(sqrt(omega_ratio_drewsen2(r_l2)),r_l2)
grid()

1.19928262204975 1.166842105263158


/tmp/ipykernel_917878/581167656.py:12: RuntimeWarning: invalid value encountered in true_divide
  ratio_w = omega_z_2[4:]/omega_r_2[4:]


In [210]:
# Display parameters of simus
# in nice table

print('|   cond  | N  | q_x |   f_x   |   f_z   |   ~a_x  | R/L | wz2/wr2 | beta|',end='\n')
print('|---------|----|-----|---------|---------|---------|-----|---------|-----|',end='\n')

for cond in range(len(file_path)):
    #print(r2_v2_rlim[myRF][k+6][50]*1000) # [condition][dim][time]
    my_try = int(file_path[cond][file_path[cond].find('DC')+2:file_path[cond].find('DC')+4])
    
    print(f'|DC{my_try:02d}_RF{my_try:02d}|{N_ions[cond]:04d}|{q[my_try]:.3f}|{sqrt(omega_x_2)[my_try]/2/pi:.3e}|{sqrt(omega_z_2)[my_try]/2/pi:.3e}|{omega_z_2[my_try]/Omega**2*2:.3e}|{alpha[cond]:.3f}|{omega_z_2[my_try]/omega_r_2[my_try]:09.06f}|{beta_guess[my_try]:.3f}|',end='\n')

|   cond  | N  | q_x |   f_x   |   f_z   |   ~a_x  | R/L | wz2/wr2 | beta|
|---------|----|-----|---------|---------|---------|-----|---------|-----|
|DC05_RF05|0256|0.540|3.676e+05|3.227e+05|5.207e-02|1.099|01.253628|0.368|
|DC06_RF06|0256|0.560|3.830e+05|3.356e+05|5.631e-02|1.125|01.246121|0.383|
|DC07_RF07|0256|0.580|3.986e+05|3.486e+05|6.076e-02|1.100|01.238061|0.399|
|DC08_RF08|0256|0.600|4.145e+05|3.617e+05|6.542e-02|1.120|01.229403|0.415|
|DC09_RF09|0256|0.620|4.307e+05|3.749e+05|7.029e-02|1.126|01.220095|0.431|
|DC10_RF10|0256|0.640|4.472e+05|3.883e+05|7.540e-02|1.143|01.210077|0.447|
|DC11_RF11|0256|0.660|4.641e+05|4.018e+05|8.073e-02|1.161|01.199283|0.464|
|DC12_RF12|0256|0.680|4.813e+05|4.155e+05|8.632e-02|1.169|01.187633|0.481|
|DC13_RF13|0256|0.700|4.990e+05|4.293e+05|9.215e-02|1.176|01.175040|0.499|
|DC03_RF03|0512|0.000|0.000e+00|0.000e+00|0.000e+00|1.095|000000nan|0.000|
|DC04_RF04|0512|0.400|2.645e+05|2.389e+05|2.853e-02|1.077|01.376719|0.265|
|DC05_RF05|0512|0.540|3.6

/tmp/ipykernel_917878/1429586881.py:11: RuntimeWarning: invalid value encountered in double_scalars
  print(f'|DC{my_try:02d}_RF{my_try:02d}|{N_ions[cond]:04d}|{q[my_try]:.3f}|{sqrt(omega_x_2)[my_try]/2/pi:.3e}|{sqrt(omega_z_2)[my_try]/2/pi:.3e}|{omega_z_2[my_try]/Omega**2*2:.3e}|{alpha[cond]:.3f}|{omega_z_2[my_try]/omega_r_2[my_try]:09.06f}|{beta_guess[my_try]:.3f}|',end='\n')


In [288]:
omega_z_2

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.25263851e+12, 4.11164456e+12, 4.44639491e+12, 4.79735431e+12,
       5.16505314e+12, 5.55006081e+12, 5.95298870e+12, 6.37449344e+12,
       6.81528055e+12, 7.27610828e+12, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.31654682e+12,
       5.84280581e+12, 5.90202343e+12, 5.94939753e+12, 5.99677163e+12,
       6.04414574e+12, 6.14284178e+12, 6.19021588e+12, 6.24153782e+12,
       6.29285977e+12, 5.87833638e+12, 5.92571048e+12, 5.97308458e+12,
       6.02045868e+12, 6.06783279e+12, 6.11915473e+12, 6.16652883e+12,
       6.21785077e+12, 6.26522487e+12, 6.31654682e+12])

In [290]:
r_l

array([1.        , 1.002002  , 1.004004  , 1.00600601, 1.00800801,
       1.01001001, 1.01201201, 1.01401401, 1.01601602, 1.01801802,
       1.02002002, 1.02202202, 1.02402402, 1.02602603, 1.02802803,
       1.03003003, 1.03203203, 1.03403403, 1.03603604, 1.03803804,
       1.04004004, 1.04204204, 1.04404404, 1.04604605, 1.04804805,
       1.05005005, 1.05205205, 1.05405405, 1.05605606, 1.05805806,
       1.06006006, 1.06206206, 1.06406406, 1.06606607, 1.06806807,
       1.07007007, 1.07207207, 1.07407407, 1.07607608, 1.07807808,
       1.08008008, 1.08208208, 1.08408408, 1.08608609, 1.08808809,
       1.09009009, 1.09209209, 1.09409409, 1.0960961 , 1.0980981 ,
       1.1001001 , 1.1021021 , 1.1041041 , 1.10610611, 1.10810811,
       1.11011011, 1.11211211, 1.11411411, 1.11611612, 1.11811812,
       1.12012012, 1.12212212, 1.12412412, 1.12612613, 1.12812813,
       1.13013013, 1.13213213, 1.13413413, 1.13613614, 1.13813814,
       1.14014014, 1.14214214, 1.14414414, 1.14614615, 1.14814

In [309]:
abs(ratio_w[7] - omega_ratio_drewsen(r_l))

/tmp/ipykernel_917878/1850721331.py:20: RuntimeWarning: invalid value encountered in true_divide
  return -2* ( arcsin(alpha_diff) - alpha*alpha_diff ) / ( arcsin(alpha_diff) - 1/alpha*alpha_diff )


array([           nan, 1.96880014e-01, 1.94476995e-01, 1.92073568e-01,
       1.89669733e-01, 1.87265493e-01, 1.84860850e-01, 1.82455805e-01,
       1.80050359e-01, 1.77644515e-01, 1.75238275e-01, 1.72831639e-01,
       1.70424609e-01, 1.68017188e-01, 1.65609377e-01, 1.63201177e-01,
       1.60792590e-01, 1.58383618e-01, 1.55974262e-01, 1.53564524e-01,
       1.51154405e-01, 1.48743908e-01, 1.46333033e-01, 1.43921782e-01,
       1.41510157e-01, 1.39098159e-01, 1.36685790e-01, 1.34273051e-01,
       1.31859944e-01, 1.29446470e-01, 1.27032631e-01, 1.24618429e-01,
       1.22203864e-01, 1.19788938e-01, 1.17373653e-01, 1.14958011e-01,
       1.12542012e-01, 1.10125658e-01, 1.07708951e-01, 1.05291891e-01,
       1.02874481e-01, 1.00456722e-01, 9.80386150e-02, 9.56201615e-02,
       9.32013630e-02, 9.07822209e-02, 8.83627364e-02, 8.59429110e-02,
       8.35227461e-02, 8.11022430e-02, 7.86814030e-02, 7.62602275e-02,
       7.38387178e-02, 7.14168753e-02, 6.89947012e-02, 6.65721968e-02,
      

In [234]:
# write parameters in a file
with open('data_parameters.txt','w') as f:
    f.write('2022 oct 20\n')
    f.write('Paramètres des simulations de chauffage RF\n')
    f.write('Simulations dans le dossier Rivendel/Simulations/20220617\n\n')
    
    f.write('|   cond  | N  | q_x|   f_x   |   f_z   |   ~a_x  | R/L | beta|\n')
    f.write('|---------|----|----|---------|---------|---------|-----|-----|\n')

    for cond in range(len(file_path)):
        #print(r2_v2_rlim[myRF][k+6][50]*1000) # [condition][dim][time]
        my_try = int(file_path[cond][file_path[cond].find('DC')+2:file_path[cond].find('DC')+4])

        f.write(f'|DC{my_try:02d}_RF{my_try:02d}|{N_ions[cond]:04d}|{q[my_try]:.2f}|{sqrt(omega_x_2)[my_try]/2/pi:.3e}|{sqrt(omega_z_2)[my_try]/2/pi:.3e}|{omega_z_2[my_try]/Omega**2*2:.3e}|{alpha[cond]:.3f}|{beta_guess[my_try]:.3f}|\n')

In [357]:
# plot temperature with fit

start_for_fit = 200
T_crit = 0.5
print('file to analyze')

to_fit = 21+2 #11 19 8
temp_to_end = 100

my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
print('>',to_fit,file_path[to_fit])
x  = time[to_fit]-1e-3
y  = T_aux_avg[to_fit]

max_f_r = sqrt(omega_r_2[my_try])/2/pi
b, a = signal.butter(8,  max_f_r/fs*2.01)
y_filt = signal.filtfilt(b, a, y, padlen=150)

index_100K = argmin(abs(y_filt-temp_to_end))
end_fit = index_100K

t0_first_guess = x[where(min(abs(y_filt-T_crit))==abs(y_filt-T_crit))][0] # 2.5e-3
B0 = log(T_crit) # log10( y[where(min(abs(y-T_crit))==abs(y-T_crit))][0] )
A0 = log(max(y_filt[start_for_fit:end_fit]) - B0)

p0 = [t0_first_guess,A0,B0,0.01,0.3]

# proper fit of the smoothed curve limiter up to threshold T
popt, pcov = curve_fit(func5, x[start_for_fit:end_fit],
                              log(y_filt[start_for_fit:end_fit]),
                       p0, maxfev= 25000,
                       bounds=((-np.inf,-np.inf,-np.inf,0,0),(np.inf,np.inf,2,1.01,1)))
print('func5',popt)

figname = 'testfitinlin0'
figure(figname,clear='True')
#xlim(0.8,6)
ax1 = subplot(111)
ax1.grid()
ax1.semilogy(x*1e3, y ,label=r'$T$',color='C0',lw=0.3)
ax1.semilogy(x*1e3, y_filt ,label=r'$T$ filtered',color='C1',ls=':',lw=3)
# ax2.vlines(x[end_fit]*1e3,1e-2,150,color='k')

y_fit5 = func5(x[start_for_fit:end_fit],*popt)
ax1.semilogy(x[start_for_fit:end_fit]*1e3,exp(y_fit5),color='r',ls='--',label=r'$\psi$',lw=2)

nticks = 9
maj_loc = ticker.LogLocator(numticks=nticks)
min_loc = ticker.LogLocator(subs='all', numticks=nticks)
ax1.yaxis.set_major_locator(maj_loc)
ax1.yaxis.set_minor_locator(min_loc)


to_fit = 1+2 #11 19 8
temp_to_end = 100
my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
print('>',to_fit,file_path[to_fit])
x  = time[to_fit]-1e-3
y  = T_aux_avg[to_fit]
max_f_r = sqrt(omega_r_2[my_try])/2/pi
b, a = signal.butter(8,  max_f_r/fs*2.01)
y_filt = signal.filtfilt(b, a, y, padlen=150)
index_100K = argmin(abs(y_filt-temp_to_end))
end_fit = index_100K
t0_first_guess = x[where(min(abs(y_filt-T_crit))==abs(y_filt-T_crit))][0] # 2.5e-3
B0 = log(T_crit) # log10( y[where(min(abs(y-T_crit))==abs(y-T_crit))][0] )
A0 = log(max(y_filt[start_for_fit:end_fit]) - B0)
p0 = [t0_first_guess,A0,B0,0.01,0.3]

# proper fit of the smoothed curve limiter up to threshold T
popt, pcov = curve_fit(func5, x[start_for_fit:end_fit],
                              log(y_filt[start_for_fit:end_fit]),
                       p0, maxfev= 25000,
                       bounds=((-np.inf,-np.inf,-np.inf,0,0),(np.inf,np.inf,2,1.01,1)))
ax1.semilogy(x*1e3, y ,label=r'$T$',color='C0',lw=0.3)
ax1.semilogy(x*1e3, y_filt ,label=r'$T$ filtered',color='C1',ls=':',lw=3)
y_fit5 = func5(x[start_for_fit:end_fit],*popt)
ax1.semilogy(x[start_for_fit:end_fit]*1e3,exp(y_fit5),color='r',ls='--',label=r'$\psi$',lw=2)

ax1.text(6,12, '$N=1024$', fontsize=30, ha='right')
ax1.text(27,12, '$N=256$', fontsize=30, ha='right')

ax1.set_xlabel('t (ms)')
ax1.set_ylabel('T (K)')
ax1.set_xlim(-1,37)
ax1.set_ylim(8e-4,510)
tight_layout()
subplots_adjust(top=0.96,
bottom=0.18)

# ax2 = subplot(212)
# ax2.grid()
# ax2.plot(x*1e3, y ,label=r'$T$',color='C0',lw=0.3)
# ax2.plot(x*1e3, y_filt ,label=r'$T$ filtered',color='C1',ls=':',lw=3)
# y_fit5 = func5(x[start_for_fit:end_fit],*popt)
# ax2.plot(x[start_for_fit:end_fit]*1e3,10**y_fit5,color='r',ls='--',label=r'$\psi$',lw=2)
# ax2.set_ylim(-10,175)
# ax2.vlines(x[end_fit]*1e3,1e-2,150,color='k')

tight_layout()

savefig(figname+'.eps',dpi=300)

file to analyze
> 23 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N1024_DC08_RF08.npz
func5 [6.72796961e-03 5.24129198e+00 9.00302814e-02 4.81142015e-03
 4.35122655e-01]
> 3 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz


In [18]:
# plot temperature with fit

start_for_fit = 200
T_crit = 0.5
print('file to analyze')

to_fit = 1 #11 19 8
temp_to_end = 75

my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
print('>',to_fit,file_path[to_fit])
x  = time[to_fit]-1e-3
y  = T_aux_avg[to_fit]

max_f_r = sqrt(omega_r_2[my_try])/2/pi
b, a = signal.butter(8,  max_f_r/fs*2.01)
y_filt = signal.filtfilt(b, a, y, padlen=150)

index_100K = argmin(abs(y_filt-temp_to_end))
end_fit = index_100K

t0_first_guess = x[where(min(abs(y_filt-T_crit))==abs(y_filt-T_crit))][0] # 2.5e-3
B0 = log(T_crit) # log10( y[where(min(abs(y-T_crit))==abs(y-T_crit))][0] )
A0 = log(max(y_filt[start_for_fit:end_fit]) - B0)

p0 = [t0_first_guess,A0,B0,0.01,0.3]

# proper fit of the smoothed curve limiter up to threshold T
popt, pcov = curve_fit(func5, x[start_for_fit:end_fit],
                              log(y_filt[start_for_fit:end_fit]),
                       p0, maxfev= 25000,
                       bounds=((0,-np.inf,-np.inf,0,0),(np.inf,np.inf,2,0.1,1)))
print('func5',popt)

figname = 'testfitinlin0_single'
figure(figname,clear='True')
#xlim(0.8,6)
ax1 = subplot(111)
ax1.grid()
ax1.semilogy(x*1e3, y ,label=r'$T$',color='C0',lw=0.3)
ax1.semilogy(x*1e3, y_filt ,label=r'$T$ filtered',color='C1',ls=':',lw=3)
# ax2.vlines(x[end_fit]*1e3,1e-2,150,color='k')

y_fit5 = func5(x[start_for_fit:end_fit],*popt)
ax1.semilogy(x[start_for_fit:end_fit]*1e3,exp(y_fit5),color='r',ls='--',label=r'$\psi$',lw=2)

nticks = 9
maj_loc = ticker.LogLocator(numticks=nticks)
min_loc = ticker.LogLocator(subs='all', numticks=nticks)
ax1.yaxis.set_major_locator(maj_loc)
ax1.yaxis.set_minor_locator(min_loc)

tight_layout()

# savefig(figname+'.eps',dpi=300)

file to analyze
> 1 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC06_RF06.npz
func5 [ 0.18592512  9.34957502 -0.0384402   0.05265561  0.11790705]


In [12]:
# Fit all curves
# Smooth data temperature
# guess first parameter set
# fit smoothed data up to threshold temperature
# much more consistent than below

popt_smooth = np.zeros((len(file_path),5))
temp_to_end = [100, 100, 75, 50, 50, 50, 50, 50, 50,
               75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75,
               100, 100, 100, 100, 100, 100, 100, 100, 75, 150, 100,
                    100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100] # Temperature to stop the fit
# 100, 100, 100, 100, 100, 100, 100, 100, 100   75, 75, 75, 75, 75, 75, 75, 75, 75
start_for_fit = 10
T_crit = 0.5
print('file to analyze')
not_fitted = 0
for to_fit in range(len(file_path)):
    j = file_path[to_fit]
# to_fit = 30
# j = file_path[to_fit]
    if to_fit!=0 and to_fit!=9 and to_fit!=10 and to_fit!=11:
    # retrive time and temperature from the curve
        print('>',to_fit,j)
        my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
        x  = time[to_fit]-1e-3
        y  = T_aux_avg[to_fit]

        # Smoothing signal with zero delay shift fit
        max_f_r = sqrt(omega_r_2[my_try])/2/pi
        b, a = signal.butter(8,  max_f_r/fs*2.01)
        y_filt = signal.filtfilt(b, a, y, padlen=150)
        
        # select index of temp where stop the fit
        # I stop the fit when T reaches a given T
        index_100K = argmin(abs(y_filt-temp_to_end[to_fit]))
        end_fit = index_100K

        # Fit parameters first guesses
        t0_first_guess = x[where(min(abs(y_filt-T_crit))==abs(y_filt-T_crit))][0] # 2.5e-3
        B0 = log(T_crit) # log10( y[where(min(abs(y-T_crit))==abs(y-T_crit))][0] )
        A0 = log(max(y_filt[start_for_fit:end_fit]) - B0)

        # set initial guess of fit parameters
        p0 = [t0_first_guess,A0,B0,0.01,0.3]
        try:
            # proper fit of the smoothed curve limiter up to threshold T
            popt, pcov = curve_fit(func5, x[start_for_fit:end_fit],
                                          log(y_filt[start_for_fit:end_fit]),
                                   p0, maxfev= 25000,
                                   bounds=((-np.inf,-np.inf,-np.inf,0,0),(np.inf,np.inf,2,1.01,1)))
            print(popt)
            popt_smooth[to_fit,:] = popt
        except RuntimeError:
            print('Optimal parameters not found: Number of calls to function has reached maxfev')
            not_fitted += 1
    else:
        print('>',to_fit,j)
        print('Data skipped')
        not_fitted += 1
    #     if to_fit == 28:
    #         tm.sleep(5)
    #     else:
    #         tm.sleep(0.5)
print('total amount of non fitted curves',not_fitted)

file to analyze
> 0 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC05_RF05.npz
Data skipped
> 1 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC06_RF06.npz
[0.18592611 8.83284272 0.0124883  0.04991899 0.13048593]
> 2 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz
[ 0.06601461  9.07674491 -0.01376579  0.04832289  0.13334638]
> 3 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz
[ 0.02775709  8.13558087 -0.16029604  0.0406409   0.16019723]
> 4 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC09_RF09.npz
[ 5.35610787e-04  4.93224454e+00 -9.57801274e-01  6.09262610e-05
  9.64952159e-01]
> 5 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC10_RF10.npz
[ 3.28762976e-03  8.38981625e+00 -4.81565026e-01  3.72296380e-02
  1.81841870e-01]
> 6 /home/adrien/RemoteFS/Rivendel/Simulat

In [13]:
start_for_fit = 200
T_crit = 0.5
print('file to analyze')
not_fitted = 0
# 29 and 30
to_fit = 34
temp_to_end = [100, 100, 75, 50, 50, 50, 50, 50, 50,
               75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75,
               100, 100, 100, 100, 100, 100, 100, 100, 75, 150, 100, 100, 100, 100, 100]
j = file_path[to_fit]
# retrive time and temperature from the curve
print('>',to_fit,j)
my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
x  = time[to_fit]-1e-3
y  = T_aux_avg[to_fit]

# Smoothing signal with zero delay shift fit
max_f_r = sqrt(omega_r_2[my_try])/2/pi
b, a = signal.butter(8,  max_f_r/fs*1.01)
y_filt = signal.filtfilt(b, a, y, padlen=150)

# Fit parameters first guesses
t0_first_guess = x[where(min(abs(y_filt-T_crit))==abs(y_filt-T_crit))][0] # 2.5e-3
B0 = log(T_crit) # log10( y[where(min(abs(y-T_crit))==abs(y-T_crit))][0] )
A0 = log(max(y_filt[start_for_fit:end_fit]) - B0)

# select index of temp where stop the fit
# I stop the fit when T reaches a given T
index_100K = argmin(abs(y_filt-temp_to_end[to_fit]))
end_fit = index_100K

# set initial guess of fit parameters
p0 = [t0_first_guess,A0,B0,0.01,0.3]

# proper fit of the smoothed curve limiter up to threshold T
popt, pcov = curve_fit(func5, x[start_for_fit:end_fit],
                              log(y_filt[start_for_fit:end_fit]),
                       p0, maxfev= 25000,
                       bounds=((-np.inf,-np.inf,-np.inf,0,0),(np.inf,np.inf,2,1.01,1)))
print(popt)
popt_smooth[to_fit,:] = popt

file to analyze
> 34 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N1024_DC24_RF24.npz
[ 1.24461362e-03  6.18743360e+00 -1.54961585e-02  1.24191799e-02
  3.25081747e-01]


In [14]:
# when does fit crosses a given temperature
# compute delta T between this moment and t0
# computed above in the fit, from t=0
T_thre = 1e-2
fit_above_thresh = np.zeros((len(file_path)))
for to_fit in range(len(file_path)):
    j = file_path[to_fit]
#     print(j)
    if to_fit!=0 and to_fit!=9 and to_fit!=10 and to_fit!=11:
        x = time[to_fit]-1e-3
        y_fit = func5(x,*popt_smooth[to_fit,:])
        fit_above_thresh[to_fit] = x[where(min(abs(exp(y_fit)-T_thre))==abs(exp(y_fit)-T_thre))][0]
# print(fit_above_thresh)

delta_T_explosion = popt_smooth[:,0] - fit_above_thresh
print(delta_T_explosion)

[0.         0.01262248 0.00621949 0.00326446 0.00013199 0.00074651
 0.0004425  0.00022652 0.00015156 0.         0.         0.
 0.01195598 0.00589594 0.00302399 0.000727   0.00084746 0.00047832
 0.00030196 0.00021851 0.02312047 0.01245747 0.00751997 0.00476984
 0.00105646 0.00115794 0.00070774 0.00049698 0.00036499 0.00137545
 0.00314277 0.00087048 0.00284347 0.00246396 0.00124399 0.00172499
 0.00033849 0.00191748 0.00182337 0.00323833 0.00093648 0.00281168
 0.00206999 0.00186097 0.00214631 0.00198269 0.00173398 0.0017573
 0.00178739]


In [126]:
popt_rework = [popt_smooth[1:9],
              popt_smooth[12:20], # 7:15
              popt_smooth[20:]]

delta_T_explosion_rework = array([delta_T_explosion[1:9],
                            delta_T_explosion[12:20], # 7:15
                            delta_T_explosion[20:]])

#  3     4     5     6     7     8     9     10    11    12    13
# 31.0, 42.0, 55.4, 57.4, 59.5, 61.5, 63.6, 65.6, 67.7, 69.7, 71.8
#             0.54  0.56  0.58  0.6   0.62  0.64  0.66  0.68  0.7
Urf_rework = [[57.4, 59.5, 61.5, 63.6, 65.6, 67.7, 69.7, 71.8],
             [57.4, 59.5, 61.5, 63.6, 65.6, 67.7, 69.7, 71.8], # 57.4, 59.5, 
             [55.4, 57.4, 59.5, 61.5, 63.6, 65.6, 67.7, 69.7, 71.8,
                      64.6, 62.5, 62.83, 63.03, 63.24, 63.44, 63.86, 64.06, 64.27, 64.47,
                      62.7262, 62.93, 63.14, 63.34, 63.55, 63.75, 63.96, 64.16, 64.37, 65.74]]
q_rework = [[0.56,  0.58,  0.6,   0.62,  0.64,  0.66,  0.68,  0.7],
             [0.56,  0.58,  0.6,   0.62,  0.64,  0.66,  0.68,  0.7], # 57.4, 59.5, 
             [0.54, 0.56,  0.58,  0.6,   0.62,  0.64,  0.66,  0.68,  0.7,
                      0.63, 0.61, 0.612, 0.614, 0.616, 0.618, 0.622, 0.624, 0.626, 0.628,
                      0.611, 0.613, 0.615, 0.617, 0.619, 0.621, 0.623, 0.625, 0.627, 0.629]]
N_rework = [256, 512, 1024]

for k,l in enumerate(N_rework):
    print(f'fit_params_{l:04d}ions')
    np.savez( f'fit_params_{l:04d}ions', fit_param=popt_rework[k], Urf=Urf_rework[k], N=N_rework[k])

# func5(t, t0, A, B, C, k)
# with load('/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/fit_params_0256ions.npz') as data:
#     a = data['N']
# print(a)

fit_params_0256ions
fit_params_0512ions
fit_params_1024ions


/tmp/ipykernel_917878/3962614403.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  delta_T_explosion_rework = array([delta_T_explosion[1:9],


In [111]:
figure('fit_parameters',clear=True)
ax1 = subplot(151)
for k in range(3):
    print(k)
    ax1.plot(Urf_rework[k],array(popt_rework[k])[:,0]*1e3-1,color=cm(k/10),ls=':',marker='P',label=N_rework[k])
    ax1.set_xlabel('$U_{RF}$ [V]')
    ax1.set_ylabel('$t_0$ [ms]')
    ax1.grid()
    ax1.legend(title='N',fontsize=12)
    ax2 = subplot(152)
    ax2.plot(Urf_rework[k],exp(array(popt_rework[k])[:,1]),color=cm(k/10),ls=':',marker='P',label=N_rework[k])
    ax2.set_xlabel('$U_{RF}$ [V]')
    ax2.set_ylabel('$e^A$ [K]')
    ax2.grid()
    ax2.legend(title='N',fontsize=12)
    ax3 = subplot(153)
    ax3.plot(Urf_rework[k],exp(array(popt_rework[k])[:,2]),color=cm(k/10),ls=':',marker='P',label=N_rework[k])
    ax3.set_xlabel('$U_{RF}$ [V]')
    ax3.set_ylabel('$e^B$ [K]')
    ax3.grid()
    ax3.legend(title='N',fontsize=12)
    ax4 = subplot(154)
    ax4.plot(Urf_rework[k],array(popt_rework[k])[:,3]*1e3,color=cm(k/10),ls=':',marker='P',label=N_rework[k])
    ax4.set_xlabel('$U_{RF}$ [V]')
    ax4.set_ylabel('$C$ [ms]')
    ax4.grid()
    ax4.legend(title='N',fontsize=12)
    ax5 = subplot(155)
    ax5.plot(Urf_rework[k],array(popt_rework[k])[:,4],color=cm(k/10),ls=':',marker='P',label=N_rework[k])
    ax5.set_xlabel('$U_{RF}$ [V]')
    ax5.set_ylabel('$k$ [1]')
    ax5.grid()
    ax5.legend(title='N',fontsize=12)
tight_layout()

0
1
2


/tmp/ipykernel_917878/223354268.py:11: RuntimeWarning: overflow encountered in exp
  ax2.plot(Urf_rework[k],exp(array(popt_rework[k])[:,1]),color=cm(k/10),ls=':',marker='P',label=N_rework[k])


In [20]:
popt_smooth[:,0]*1e3,delta_T_explosion_rework[k]*1e3

(array([  0.        , 185.92610854,  66.01461186,  27.75708798,
          0.53561079,   3.28762976,   1.58462496,   0.84113981,
          0.45018168,   0.        ,   0.        ,   0.        ,
         75.19160449,  29.08405997,  12.3071152 ,   1.11312209,
          2.0340864 ,   1.17444141,   0.67958333,   0.38113077,
         78.87959681,  31.67659639,  14.95609762,   6.72796534,
          1.05707995,   1.49755958,   0.85936892,   0.5721043 ,
          0.36561028,   1.80657887,   4.28239016,   1.1066055 ,
          3.47009572,   3.20708665,   1.24461362,   1.72561337,
          0.66961102,   2.2866081 ,   2.14249791,   3.98795431,
          1.07610832,   3.52780198,   2.3106147 ,   1.86159569,
          2.69092994,   2.41181692,   1.95110578,   2.13592494,
          1.78801363]),
 array([23.12047231, 12.45747189,  7.51997312,  4.76984084,  1.05645545,
         1.15793508,  0.70774442,  0.4969798 ,  0.36498578,  1.37545437,
         3.14276566,  0.870481  ,  2.84347122,  2.46396215,  1

In [385]:
C = []
D = []
x_forfit = [linspace(0.555,0.71,200),linspace(0.555,0.71,200),linspace(0.535,0.71,200)]
figname = 'fit_parameter_t0_log'
fig = figure(figname,clear=True)
fig.set_size_inches(11.7, 8.3)
ax1 = subplot(111)
for k in range(1):
    ax1.loglog(q_rework[k],delta_T_explosion_rework[k]*1e3,
                 color=cm(k/10),ls='',marker='v',ms=17,mew=0,label=N_rework[k])
    xx = log(hstack((q_rework[k][:3],q_rework[k][4:])))
    yy = log(hstack((delta_T_explosion_rework[k][:3],delta_T_explosion_rework[k][4:])))
    popt, pcov = curve_fit(func_lin, array(xx),yy,maxfev= 5000)
    print(k,popt)
    C.append(exp(popt[1]))
    D.append(popt[0])
    ax1.loglog(x_forfit[k],func_pow2(x_forfit[k],popt[0],popt[1])*1e3,
                 ls='--',lw=2,color=cm(k/10))
    
for k in range(1,2):
    ax1.loglog(q_rework[k],delta_T_explosion_rework[k]*1e3,
                 color=cm(k/10),ls='',marker='D',ms=13,mew=0,label=N_rework[k])
    xx = log(hstack((q_rework[k][:3],q_rework[k][4:])))
    yy = log(hstack((delta_T_explosion_rework[k][:3],delta_T_explosion_rework[k][4:])))
    popt, pcov = curve_fit(func_lin, array(xx),yy,maxfev= 5000)
    print(k,popt)
    C.append(exp(popt[1]))
    D.append(popt[0])
    ax1.loglog(x_forfit[k],func_pow2(x_forfit[k],popt[0],popt[1])*1e3,
                 ls='--',lw=2,color=cm(k/10))
    
for k in range(2,3):
    ax1.loglog(q_rework[k][:9],delta_T_explosion_rework[k][:9]*1e3,
                 color=cm(k/10),ls='',marker='P',ms=17,mew=0,label=N_rework[k])
    xx = log(hstack((q_rework[k][:4],q_rework[k][5:9])))
    yy = log(hstack((delta_T_explosion_rework[k][:4],delta_T_explosion_rework[k][5:9])))
    popt, pcov = curve_fit(func_lin, array(xx),yy,maxfev= 5000)
    print(k,popt)
    C.append(exp(popt[1]))
    D.append(popt[0])
    ax1.loglog(x_forfit[k],func_pow2(x_forfit[k],popt[0],popt[1])*1e3,
                 ls='--',lw=2,color=cm(k/10))
    
ax1.set_xlabel('$q_{x}$ (log scale)')
ax1.set_ylabel('$\Delta t$ (ms)')
ax1.grid()
    
# k = 2
# ax1.loglog(q_rework[2][:9],delta_T_explosion_rework[2][:9]*1e3,
#                  color=cm(2/10),ls='',marker='P',ms=17,mew=0,label=N_rework[2])
# popt, pcov = curve_fit(func_lin, log10(array(q_rework[2][:9])),
#                        log10(delta_T_explosion_rework[2][:9]),
#                        maxfev= 5000)
# print(k,popt)
# ax1.loglog(x_forfit[2],func_pow(x_forfit[2],popt[0],popt[1])*1e3,
#                  ls='--',lw=2,color=cm(2/10))
ax1.set_xticks([0.54,0.56,0.58,0.6,0.62,0.64,0.66,0.68,0.7])
# ax1.set_xticklabels(rotation=45, ha='right')
ax1.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax1.grid(visible=True, which='major', color='0.2', linestyle='-')
ax1.grid(visible=True, which='minor', color='0.7', linestyle='--')
# ax1.legend(title='N',fontsize=18)
tight_layout()

savefig(figname+'.eps',dpi=300)
# savefig(figname+'.png',dpi=300)
    
# fig = figure('fit_parameter_t0_bis',clear=True)
# fig.set_size_inches(11.7, 8.3)
# ax1 = subplot(111)
# for k in range(3):
#     ax1.plot(array(q_rework[k]),(array(popt_rework[k])[:,0]*1e3)**(-1/2),color=cm(k/4),
#              ls=':',marker='P',ms=10,label=N_rework[k])
#     popt, pcov = curve_fit(func_lin, array(q_rework[k]),(array(popt_rework[k])[:,0]*1e3)**(-1/2),maxfev= 15000)
#     ax1.plot(array(q_rework[k]),func_lin(array(q_rework[k]),popt[0],popt[1]),color=cm(k/3))
#     ax1.set_xlabel('$q_x$')
#     ax1.set_ylabel('$t_0^{-1/2}$ [ms$^{-1/2}$]')
#     ax1.grid()
#     ax1.legend(title='N',fontsize=12)
# tight_layout()

0 [-20.30085491 -16.15032485]
1 [-18.38802173 -15.16883067]
2 [-16.68375342 -14.03159614]


In [387]:
# Around specific values

k = 2
x = hstack((q_rework[k][4],q_rework[k][9:]))
y = hstack((delta_T_explosion_rework[k][4]*1e3,delta_T_explosion_rework[k][9:]*1e3))

x, y = zip(*sorted(zip(x, y)))
figname='fit_parameter_t0_zoom'
fig = figure(figname,clear=True)
fig.set_size_inches(11.7, 8.3)
ax1 = subplot(111)
ax1.plot(x,y,color=cm(k/10),ls=':',marker='P',ms=17,mew=1.5,label=N_rework[k])

ax1.plot(linspace(0.6095,0.6305,200),func_pow2(linspace(0.6095,0.6305,200),D[2],log(C[2]))*1e3,
                 ls='--',lw=4,color='k')
ticks = np.linspace(0.61,0.63,11)
ax1.set_xticks(ticks)
ax1.xaxis.set_minor_formatter(ticker.ScalarFormatter())
plt.xticks(rotation = 45)
ax1.set_xlabel('$q_{x}$')
ax1.set_ylabel('$\Delta t$ (ms)')
ax1.grid()
# ax1.legend(title='N',fontsize=18)
tight_layout()
savefig(figname+'.eps',dpi=300)

In [23]:
# fig = figure('fit_parameter_B',clear=True)
# fig.set_size_inches(11.7, 8.3)
# ax1 = subplot(111)
# for k in range(2):
#     ax1.plot(Urf_rework[k],array(popt_rework[k])[:,2]*1e3,color=cm(k/5),ls=':',marker='P',ms=10,mew=1,label=N_rework[k])
#     ax1.set_xlabel('$U_{RF}$ [V]')
#     ax1.set_ylabel('$t_0$ [ms]')
#     ax1.grid()
#     ax1.legend(title='N',fontsize=12)
# ax1.plot(Urf_rework[2],array(popt_rework[2])[:,2]*1e3,color=cm(2/5),ls=':',marker='P',ms=10,mew=1,label=N_rework[2])
# tight_layout()
    
figname = 'fit_parameter_B_bis'
fig = figure(figname,clear=True)
fig.set_size_inches(11.7, 8.3)
ax1 = subplot(111)
for k in range(3):
    ax1.plot(array(q_rework[k]),10**array(popt_rework[k])[:,2],
             color=cm(k/10),ls='',marker='P',ms=17,mew=0,label=N_rework[k])
    xx = hstack((q_rework[k][:3],q_rework[k][4:]))
    yy = hstack((array(popt_rework[k])[:3,2],array(popt_rework[k])[4:,2]))
    popt, pcov = curve_fit(func_lin, array(xx),10**yy,maxfev= 5000)
#     popt, pcov = curve_fit(func_lin, array(q_rework[k]),(10**array(popt_rework[k])[:,2]),maxfev= 15000)
    ax1.plot(array(q_rework[k]),func_lin(array(q_rework[k]),popt[0],popt[1]),
             color=cm(k/10),ls='--',lw=2)
    
# ax1.plot(array(q_rework[2][:9]),10**array(popt_rework[2][:9])[:,2],
#              color=cm(2/10),ls='',marker='P',ms=17,mew=0,label=N_rework[2])
# popt, pcov = curve_fit(func_lin, array(q_rework[2][:9]),(10**array(popt_rework[2][:9])[:,2]),maxfev= 15000)
# ax1.plot(array(q_rework[2][:9]),func_lin(array(q_rework[2][:9]),popt[0],popt[1]),
#          color=cm(2/10),ls='--',lw=2)
ax1.set_xlabel('$q_x$')
ax1.set_ylabel('$10^B$ [K]')
ax1.grid()
ax1.legend(title='N',fontsize=18)
tight_layout()
# savefig(figname+'.eps',dpi=300)

In [114]:
C = []
D = []
x_forfit = [linspace(0.555,0.71,200),linspace(0.555,0.71,200),linspace(0.535,0.71,200)]
figname = 'fit_parameter_t0_log_B_bis'
fig = figure(figname,clear=True)
fig.set_size_inches(11.7*sqrt(2), 5.8)
# fig.set_size_inches(11.7, 8.3)
ax1 = subplot(121)
for k in range(2):
    ax1.loglog(q_rework[k],delta_T_explosion_rework[k]*1e3,
                 color=cm(k/10),ls='',marker='P',ms=14,mew=0,label=N_rework[k])
    popt, pcov = curve_fit(func_lin, log10(array(q_rework[k])),log10(delta_T_explosion_rework[k]),maxfev= 5000)
    print(k,popt)
    C.append(10**popt[1])
    D.append(popt[0])
    ax1.loglog(x_forfit[k],func_pow(x_forfit[k],popt[0],popt[1])*1e3,
                 ls='--',color=cm(k/10),lw=2)
    ax1.set_xlabel('$q_{x}$ (log scale)')
    ax1.set_ylabel('$\Delta t$ [ms] (log scale)')
    ax1.grid()
    
k = 2
ax1.loglog(q_rework[2][:9],delta_T_explosion_rework[2][:9]*1e3,
                 color=cm(2/10),ls='',marker='P',ms=14,mew=0,label=N_rework[2])
popt, pcov = curve_fit(func_lin, log10(array(q_rework[2][:9])),
                       log10(delta_T_explosion_rework[2][:9]),
                       maxfev= 5000)
print(k,popt)
ax1.loglog(x_forfit[2],func_pow(x_forfit[2],popt[0],popt[1])*1e3,
                 ls='--',color=cm(2/10),lw=2)
ax1.set_xticks([0.54,0.56,0.58,0.6,0.62,0.64,0.66,0.68,0.7])
ax1.set_xticklabels([0.54,0.56,0.58,0.6,0.62,0.64,0.66,0.68,0.7],rotation=45, ha='right')
ax1.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax1.grid(visible=True, which='major', color='0.2', linestyle='-')
ax1.grid(visible=True, which='minor', color='0.7', linestyle='--')
ax1.legend(title='N',fontsize=18)
tight_layout()

ax2 = subplot(122)
for k in range(2):
    ax2.plot(array(q_rework[k]),10**array(popt_rework[k])[:,2],
             color=cm(k/10),ls='',marker='P',ms=14,mew=0,label=N_rework[k])
    popt, pcov = curve_fit(func_lin, array(q_rework[k]),(10**array(popt_rework[k])[:,2]),maxfev= 15000)
    ax2.plot(array(q_rework[k]),func_lin(array(q_rework[k]),popt[0],popt[1]),
             color=cm(k/10),ls='--',lw=2)
    
ax2.plot(array(q_rework[2][:9]),10**array(popt_rework[2][:9])[:,2],
             color=cm(2/10),ls='',marker='P',ms=14,mew=0,label=N_rework[2])
popt, pcov = curve_fit(func_lin, array(q_rework[2][:9]),(10**array(popt_rework[2][:9])[:,2]),maxfev= 15000)
ax2.plot(array(q_rework[2][:9]),func_lin(array(q_rework[2][:9]),popt[0],popt[1]),color=cm(2/10),ls='--',lw=2)
ax2.set_xlabel('$q_x$')
ax2.set_ylabel('$B=T(t=t_0)$ [K]')
ax2.set_xticks([0.54,0.56,0.58,0.6,0.62,0.64,0.66,0.68,0.7])
ax2.set_xticklabels([0.54,0.56,0.58,0.6,0.62,0.64,0.66,0.68,0.7],rotation=45, ha='right')
ax2.grid()
ax2.legend(title='N',fontsize=18)
tight_layout()
# savefig(figname+'.eps',dpi=300)

0 [-19.52258098  -6.99194312]
1 [-18.12268838  -6.58022074]
2 [-16.72786227  -6.14151232]


In [24]:
# Around specific values

k = 2
xt = hstack((q_rework[k][4],q_rework[k][9:]))
yt = hstack((delta_T_explosion_rework[k][4]*1e3,delta_T_explosion_rework[k][9:]*1e3))
xb = hstack((q_rework[k][4],q_rework[k][9:]))
yb = hstack((10**array(popt_rework[k])[:,2][4],10**array(popt_rework[k])[:,2][9:]))
dummyx = linspace(0.61,0.63,200)

xt, yt = zip(*sorted(zip(xt, yt)))
xb, yb = zip(*sorted(zip(xb, yb)))
figname='fit_parameter_B_t0_zoom'
fig = figure(figname,clear=True)
fig.set_size_inches(11.7, 8.3)
ax1 = subplot(211)
ax1.plot(xt,yt,color=cm(k/10),ls=':',marker='P',ms=17,mew=0,label=N_rework[k])
popt, pcov = curve_fit(func_lin, log10(array(q_rework[k][:9])),
                       log10(delta_T_explosion_rework[k][:9]),
                       maxfev= 5000)
ax1.plot(dummyx,func_pow(dummyx,popt[0],popt[1])*1e3,
                 ls='--',color=cm(k/10),lw=2.5)
plt.setp(ax1.get_xticklabels(),visible=False)
ax1.set_ylabel('$\Delta t$ [ms]')
ax1.grid()
ax1.legend(title='N',fontsize=18)


ax2 = subplot(212,sharex=ax1)

ax2.plot(xb,yb,color=cm(k/10),ls=':',marker='P',ms=17,mew=0,label=N_rework[2])
popt, pcov = curve_fit(func_lin, array(q_rework[k][:9]),(10**array(popt_rework[k][:9])[:,2]),maxfev= 15000)
ax2.plot(dummyx,func_lin(dummyx,popt[0],popt[1]),
         color=cm(k/10),ls='--',lw=2.5)
ax2.set_xlabel('$q_x$')
ax2.set_ylabel('$B=T(t=t_0)$ [K]')
ax2.grid()

ticks = np.linspace(0.61,0.63,11) # hstack((0.60,np.linspace(0.61,0.63,11),0.64)) # np.linspace(0.61,0.63,11)
ax2.set_xticks(ticks)
ax2.xaxis.set_minor_formatter(ticker.ScalarFormatter())
plt.xticks(rotation = 45)
tight_layout()

subplots_adjust(hspace=0.015)
savefig(figname+'.eps',dpi=300)

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


# Windowing

In [25]:
c_light = 299792458
h_pl = 6.62607015*1e-34
h_pl_bar = h_pl/2/pi
lam_397 = 396.84620*1e-9
k=(2*pi)/lam_397
gamma_SP = 21.57*1e6
tau = 1/gamma_SP
Omega = 2.0e6*2*pi
tau_RF = 1/(Omega/(2*pi))

In [26]:
# window_samp = 200
# window_size = tau_RF * window_samp
# window_num = len(y)/window_samp
# print(window_num)
# print(len(y))

In [27]:
# Windowing temperature
window_samp = 200
x_win = [] # zeros((len(y)//window_samp,len(file_path)))
y_win = [] # zeros((len(y)//window_samp,len(file_path)))

for to_fit in range(len(file_path)):
#     print(to_fit)
    x_win.append([])
    y_win.append([])
    x = time[to_fit]-1e-3
    y = T_aux_avg[to_fit]
    for i in range(len(y)//window_samp-1):
        x_win[to_fit].append( (x[i*window_samp]+x[(i+1)*window_samp])/2 )
        y_win[to_fit].append( sum(y[i*window_samp:(i+1)*window_samp])/window_samp )
    
print(diff(x_win[0])[0]*1e3,tauRF*window_samp*1e3)

0.10000000000000005 0.09999999999999999


In [122]:
# Plot windowed temperature
to_plot = 25
x  = time[to_plot]-1e-3
y  = T_aux_avg[to_plot]
figure('Windowed T',clear='True')
semilogy(x*1e3, y,':')
semilogy(array(x_win[to_plot])*1e3, y_win[to_plot],ls='',marker='+',ms=10,mew=2)
xlabel('t [ms]')
ylabel('T [K]')
# hlines(500,0,0.02)
grid()
tight_layout()

In [123]:
figname = 'Windowed_dT_dt'
fig = figure(figname,clear='True')
fig.set_size_inches(11.7, 8.3)
ax = fig.add_subplot(111)
# ax.semilogy(x_win[:-1]*1e3, diff(y_win)/window_samp,'+')
for to_plot in [0,1,2,3,4,5,6]: # [21,23,25,27] [12,14,16,18] [1,3,5,7]
    my_try = int(file_path[to_plot][file_path[to_plot].find('DC')+2:file_path[to_plot].find('DC')+4])
    print(file_path[to_plot])
    ax.loglog(y_win[to_plot][:-1], diff(y_win[to_plot])/window_samp,
              marker='.',ms=20,mec='k',mew=0.3,ls='',
              label=f'{q[my_try]}')
# semilogy(x[::100]*1e3, y[::100],':',color='y')
# ax.set_xlabel('t [ms]')
ax.set_xlabel('T [K]')
ax.set_ylabel('$H=dT/dt$ [K/RF period]')
ax.set_xlim(3e-3,11)
ax.grid()
legend(title='$q_x$, $N=1024$',loc=0)
tight_layout()
# savefig(figname+'N1024_zoom'+'.eps',dpi=300)

/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC05_RF05.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC06_RF06.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC09_RF09.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC10_RF10.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC11_RF11.npz


In [144]:
# Plot windowed temperature
# figure('Windowed T',clear='True')
# semilogy(x_win*1e3, y_win,'+')
# semilogy(x[::100]*1e3, y[::100],':',color='y')
# grid()
# tight_layout()
figname = 'Windowed_dT_dt'
fig = figure(figname,clear='True')
fig.set_size_inches(11.7, 8.3)
ax = fig.add_subplot(111)
# ax.semilogy(x_win[:-1]*1e3, diff(y_win)/window_samp,'+')
incr = 0
for to_plot in [3,14,23]: # [4,15,24]
    print(file_path[to_plot])
    my_try = int(file_path[to_plot][file_path[to_plot].find('DC')+2:file_path[to_plot].find('DC')+4])
    ax.loglog(y_win[to_plot][:-1], diff(y_win[to_plot])/window_samp,
              marker='.',ms=20,mec='k',mew=0.3,ls='',
              label=f'{N_rework[incr]}')
    incr+=1
# semilogy(x[::100]*1e3, y[::100],':',color='y')
# ax.set_xlabel('t [ms]')
ax.set_xlabel('T [K]')
ax.set_ylabel('$H=\mathrm{d}T/\mathrm{d}t$ [K/RF period]')
# ax.set_xlim(3e-3,11)
ax.grid()
legend(title=f'$N$, $q_x={q[my_try]}$',loc=4)
tight_layout()

# savefig(figname+'q0.60'+'.eps',dpi=300)

/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0512_DC08_RF08.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N1024_DC08_RF08.npz


In [359]:
# Plot windowed temperature
# figure('Windowed T',clear='True')
# semilogy(x_win*1e3, y_win,'+')
# semilogy(x[::100]*1e3, y[::100],':',color='y')
# grid()
# tight_layout

nv = 1000
log_temps = linspace(-3.5,3,nv)
coolpow_05 = []
coolpow_1 = []
# coolpow_1_bis = []
coolpow_2 = []
# coolpow_4 = []
# coolpow_6 = []
# coolpow_8 = []
# coolpow_10 = []
# coolpow_50 = []
hotpow = []
for T in 10**log_temps:
    v_thermique = sqrt(2*kb*T/m_Ca)
    dv = 10*v_thermique/nv
    coolpow_05.append( sum(cool_power(-dv*nv/2, dv*nv/2, nv, T, -Gam/2, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_1.append( sum(cool_power(-dv*nv/2, dv*nv/2, nv, T, -Gam, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_1_bis.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -Gam, k, 2*Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_2.append( sum(cool_power(-dv*nv/2, dv*nv/2, nv, T, -2*Gam, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_4.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -4*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_6.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -6*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_8.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -8*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_10.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -10*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_50.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -50*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    hotpow.append( sum(hot_power(-dv*nv/2, dv*nv/2, nv, T, -Gam/2, klam, Rab, Gam))*dv*h_pl_bar**2*klam**2/m_Ca*Gam/kb*tau_RF )

my_color=[7,2,6,3] # 
figname = 'Windowed_dT_dt_q_N'
fig = figure(figname,clear='True')
fig.set_size_inches(11.7, 8.3)
ax1 = fig.add_subplot(111)

# ax.semilogy(x_win[:-1]*1e3, diff(y_win)/window_samp,'+')
incr = 0
choosen_plot = [3,14,23]
my_marker=['v','D','P','.']
my_size = [17,13,17,17]
for to_plot in choosen_plot: # [4,15,24] 1,12,21 3,14,23
    print(file_path[to_plot])
    my_try = int(file_path[to_plot][file_path[to_plot].find('DC')+2:file_path[to_plot].find('DC')+4])
    ax1.loglog(y_win[to_plot][:-1], diff(y_win[to_plot])/window_samp,
              marker=my_marker[incr],ms=my_size[incr],mec='k',mew=0.2,ls='',color=cm(incr/8),
              label=f'{N_rework[incr]}')
    incr+=1

# to_plot = 20
# ax1.loglog(y_win[to_plot][:-1], diff(y_win[to_plot])/window_samp,
#           marker='.',ms=14,mec='k',mew=0,ls='',color=cm(4/8),
#           label=f'{N_rework[2]}')

ax1.loglog(y_win[choosen_plot[0]][265:-100],
           diff(y_win[choosen_plot[0]][265:-99])/window_samp,
          marker='',ls=':',color=cm(0/8))
ax1.loglog(y_win[choosen_plot[1]][100:-135],
           diff(y_win[choosen_plot[1]][100:-134])/window_samp,
          marker='',ls=':',color=cm(1/8))
ax1.loglog(y_win[choosen_plot[2]][60:-90],
           diff(y_win[choosen_plot[2]][60:-89])/window_samp,
          marker='',ls=':',color=cm(2/8))

# ax1.loglog(y_f,y_fit = func5(x,*popt_smooth[to_fit,:]))

howdoyouturnthison = False
if howdoyouturnthison == True:
    ax1.loglog(10**log_temps,coolpow_05,ls="-",color='xkcd:red',lw=4,label='$-0.5\Gamma$')
    ax1.loglog(10**log_temps,coolpow_1,ls="--",color='xkcd:red',lw=4,label='$-\Gamma$')
    ax1.loglog(10**log_temps[:nv-300],hotpow[:nv-300],ls='-',color='k',lw=4,label='$H_e$') # 'dashdot'
else:
    ax1.loglog(10**log_temps,coolpow_05,ls="-",color='xkcd:red',lw=4)
    ax1.loglog(10**log_temps,coolpow_1,ls="--",color='xkcd:red',lw=4)
    ax1.loglog(10**log_temps[:nv-300],hotpow[:nv-300],ls='-',color='k',lw=4) # 'dashdot'

ax1.set_xlabel('$T$ (K)')
ax1.set_ylabel(r'$\mathrm{d}T/\mathrm{d}t$ (K/$\tau_{RF}$)')
ax1.grid()
ax1.grid(True, which="minor", ls=":", color='0.80')
ax1.set_xlim(3e-3,555)
ax1.set_ylim(2e-9,0.3)
ax1.legend(title=f'$N$',loc=4,ncol=1,fontsize=22)

# old part for showing q dependency
# replaced by 20220930_Heating_rate_low_q-ForArticle

# ax2 = fig.add_subplot(122,sharey=ax1,sharex=ax1)
# # ax.semilogy(x_win[:-1]*1e3, diff(y_win)/window_samp,'+')
# incr = 0
# # ax2.loglog(y_win[49][:-1], diff(y_win[49])/window_samp,
# #               marker='.',ms=17,mec='k',mew=0.2,ls='',color=cm((5+1)/8),
# #               label=f'0.4')
# # ax2.loglog(y_win[49][len(y_win[49])//2+325:len(y_win[49])//2+350],
# #            diff(y_win[49][len(y_win[49])//2+325:len(y_win[49])//2+350+1])/window_samp,
# #           marker='',ls=':',color=cm(6/8))
# for to_plot in [21,23,25,27]: # [21,23,25,27] [12,14,16,18] [1,3,5,7]
#     my_try = int(file_path[to_plot][file_path[to_plot].find('DC')+2:file_path[to_plot].find('DC')+4])
#     print(file_path[to_plot])
#     ax2.loglog(y_win[to_plot][:-1], diff(y_win[to_plot])/window_samp,
#               marker='.',ms=17,mec='k',mew=0.2,ls='',
#               label=f'{q[my_try]}',color=cm2(my_color[incr]/8))
#     incr+=1
    
# ax2.loglog(y_win[21][len(y_win[21])//2+100:len(y_win[21])//2+120],
#            diff(y_win[21][len(y_win[21])//2+100:len(y_win[21])//2+120+1])/window_samp,
#           marker='',ls=':',color=cm2(7/8))
# ax2.loglog(y_win[23][len(y_win[23])//2-30:len(y_win[23])//2-12],
#            diff(y_win[23][len(y_win[23])//2-30:len(y_win[23])//2-12+1])/window_samp,
#           marker='',ls=':',color=cm2(2/8))
# ax2.loglog(y_win[25][len(y_win[25])//2-20:len(y_win[25])//2+1],
#            diff(y_win[25][len(y_win[25])//2-20:len(y_win[25])//2+1+1])/window_samp,
#           marker='',ls=':',color=cm2(6/8))
# ax2.loglog(y_win[27][len(y_win[27])//2-45:len(y_win[27])//2-15],
#            diff(y_win[27][len(y_win[27])//2-45:len(y_win[27])//2-15+1])/window_samp,
#           marker='',ls=':',color=cm2(3/8))

# ax2.loglog(10**log_temps,coolpow_05,ls="-",lw=2,color='k',label='-$0.5\Gamma$')
# ax2.loglog(10**log_temps,coolpow_1,ls="--",lw=2,color='k',label='-$\Gamma$')
# ax2.loglog(10**log_temps,coolpow_2,ls=":",lw=2.5,color='k',label='$-2\Gamma$')  
# ax2.loglog(10**log_temps[:nv-300],hotpow[:nv-300],linestyle='-',color='xkcd:red',lw=2.5,label='$H_e$') 
# # semilogy(x[::100]*1e3, y[::100],':',color='y')
# # ax.set_xlabel('t [ms]')
# ax2.set_xlabel('T (K)')
# ax2.grid(True, which="minor", ls=":", color='0.80')
# ax2.set_ylim(1e-9,1)
# ax2.set_xlim(3e-3,500)
# ax2.grid()
# ax2.set_yticks([1e-9,1e-8,1e-7,1e-6,1e-5,1e-4,1e-3,1e-2,1e-1,1])
# ax2.legend(title='$q_x$, $N=1024$',loc=4,ncol=2,fontsize=18)

tight_layout()

savefig(figname+'varq_varN_with0.60_guides'+'.eps',dpi=300)
# savefig(figname+'varq_varN_with0.60_guides_0.4bonus'+'.jpg',dpi=300)
# savefig(figname+'varq_varN_with0.60_guides'+'.png',dpi=300)

/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0512_DC08_RF08.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N1024_DC08_RF08.npz


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [101]:
# Plot windowed temperature
# figure('Windowed T',clear='True')
# semilogy(x_win*1e3, y_win,'+')
# semilogy(x[::100]*1e3, y[::100],':',color='y')
# grid()
# tight_layout

nv = 1000
log_temps = linspace(-3.5,3,nv)
coolpow_05 = []
coolpow_1 = []
# coolpow_1_bis = []
coolpow_2 = []
# coolpow_4 = []
# coolpow_6 = []
# coolpow_8 = []
# coolpow_10 = []
# coolpow_50 = []
hotpow = []
for T in 10**log_temps:
    v_thermique = sqrt(2*kb*T/m_Ca)
    dv = 10*v_thermique/nv
    coolpow_05.append( sum(cool_power(-dv*nv/2, dv*nv/2, nv, T, -Gam/2, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_1.append( sum(cool_power(-dv*nv/2, dv*nv/2, nv, T, -Gam, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_1_bis.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -Gam, k, 2*Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_2.append( sum(cool_power(-dv*nv/2, dv*nv/2, nv, T, -2*Gam, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_4.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -4*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_6.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -6*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_8.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -8*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_10.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -10*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_50.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -50*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    hotpow.append( sum(hot_power(-dv*nv/2, dv*nv/2, nv, T, -Gam/2, klam, Rab, Gam))*dv*h_pl_bar**2*klam**2/m_Ca*Gam/kb*tau_RF )

my_color=[7,2,6,3] # 
figname = 'Windowed_dT_dt_q_N_zoomq'
fig = figure(figname,clear='True')
fig.set_size_inches(11.7, 8.3)
ax1 = fig.add_subplot(111)

# ax.semilogy(x_win[:-1]*1e3, diff(y_win)/window_samp,'+')
incr = 0
choosen_plot = [1,30,31,40,36]
for to_plot in choosen_plot: # [4,15,24] 1,12,21 3,14,23
    print(file_path[to_plot])
    my_try = int(file_path[to_plot][file_path[to_plot].find('DC')+2:file_path[to_plot].find('DC')+4])
    ax1.loglog(y_win[to_plot][:-1], diff(y_win[to_plot])/window_samp,
              marker='.',ms=17,mec='k',mew=0.2,ls=':',color=cm(incr/8),
              label=f'{q[my_try]:.3f}')
    incr+=1

# ax1.loglog(y_f,y_fit = func5(x,*popt_smooth[to_fit,:]))

# ax1.loglog(0,0,marker='',ls='',label=' ')
    
ax1.loglog(10**log_temps,coolpow_05,ls="-",color='k',lw=2,label='$-0.5\Gamma$')
ax1.loglog(10**log_temps,coolpow_1,ls="--",color='k',lw=2,label='$-\Gamma$')
# ax1.loglog(10**log_temps,coolpow_1_bis,ls="--",color='r',label='$-\Gamma$')
ax1.loglog(10**log_temps,coolpow_2,ls=":",color='k',lw=2.5,label='$-2\Gamma$')
ax1.loglog(10**log_temps[:nv-300],hotpow[:nv-300],ls='-',color='xkcd:red',lw=2.5,label='$H_e$') # 'dashdot'


ax1.set_xlabel('T (K)')
ax1.set_ylabel('$\mathrm{d}T/\mathrm{d}t$ (K/RF period)')
ax1.grid()
ax1.grid(True, which="minor", ls=":", color='0.80')
ax1.legend(title=f'$N=1024$, $q_x$',loc=4,ncol=2,fontsize=18)

tight_layout()

# savefig(figname+'special_q'+'.eps',dpi=300)
# savefig(figname+'varq_varN_with0.60_guides_0.4bonus'+'.jpg',dpi=300)
# savefig(figname+'special_q'+'.png',dpi=300)

/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N0256_DC06_RF06.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N1024_DC20_RF20.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N1024_DC21_RF21.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N1024_DC30_RF30.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/Time_and_temp_RFHEAT_N1024_DC26_RF26.npz


# From fluo-variations_optimal-temp.ipynb

# Functions used to describe ions

$\texttt{MB}$ is the Maxwell-Boltzmann distribution. It provides the probability to find an atom with a given velocity in a gas with temperature T.

$\texttt{pfl_dop} = \rho_{ee}$ is the atomic ray profile with Doppler effect. It provides the probability of excitation given Rabi frequency, detuning, lambda and velocity of atom. It is considered equal to the excited population, i.e the proportion of excited atoms.

$ \texttt{MB} = \sqrt{\frac{m}{2\pi k_BT}}\exp{-\frac{mv^2}{2k_BT}}$

$\texttt{pfl_dop} = \rho_{ee} = \frac{0.25\Omega_R^2}{0.5\Omega_R^2 + 0.25\Gamma^2 + \Delta^2} = \frac{A/2}{A+B + \Delta^2}$

In [30]:
# Doppler profile
def pfl_dop(v, delta, k, Rab, Gam):
    return .25*Rab**2/(0.5*Rab**2+.25*Gam**2+(delta-k*v)**2)

# Maxwell-Boltzmann distribution
def MB(v, T):
    kb = 1.38064852*1e-23  # Boltzman
    m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
    return (m_Ca/(np.pi*2*kb*T))**(1/2) * np.exp(-m_Ca*v**2/(2*kb*T)) # **1 car vitesse par rapport au laser compte seulement (1D)

# Probabilities product
# For a range of velocities,
# compute the prob_fluo
# for a given T, delta, Rab, Gam
def prob_fluo(vmin, vmax, nv, T, delta, k, Rab, Gam):
    nu = k*np.linspace(vmin, vmax, nv)
    return nu, pfl_dop(-nu/k, delta, k, Rab, Gam)*MB(nu/k, T)

def cool_power(vmin, vmax, nv, T, delta, k, Rab, Gam):
    nu = k*np.linspace(vmin, vmax, nv)
    return nu, pfl_dop(-nu/k, delta, k, Rab, Gam)*MB(nu/k, T)*nu

def hot_power(vmin, vmax, nv, T, delta, k, Rab, Gam):
    nu = k*np.linspace(vmin, vmax, nv)
    return nu, pfl_dop(-nu/k, delta, k, Rab, Gam)*MB(nu/k, T)

def T_lim(delta):
    return -0.5*hbar*Gam**2*(1+(2*delta/Gam)**2) / (4*delta)/kb

In [31]:
T = 0.01
v_thermique = sqrt(2*kb*T/m_Ca)
nv = 2000
dv = 6*v_thermique/nv
dist_MB = []
for v in linspace(-3*v_thermique,3*v_thermique,nv):
    dist_MB.append(MB(v, T))
print(sum(dist_MB)*dv)

0.9994781288008786


In [32]:
# Laser parameters
lam = 397e-9 # m
klam = 2*pi/lam # m^-1
Gam = 21570000.0 *2*pi
delta = Gam
I = 170 # W/m²
Rab = Gam

# Lifetime vs Heating rate
How does lifetime t0 compares with the time when H=G ?

In [74]:
from __future__ import division
def interpolated_intercept(x, y1, y2):
    """Find the intercept of two curves, given by the same x data"""

    def intercept(point1, point2, point3, point4):
        """find the intersection between two lines
        the first line is defined by the line between point1 and point2
        the first line is defined by the line between point3 and point4
        each point is an (x,y) tuple.

        So, for example, you can find the intersection between
        intercept((0,0), (1,1), (0,1), (1,0)) = (0.5, 0.5)

        Returns: the intercept, in (x,y) format
        """    

        def line(p1, p2):
            A = (p1[1] - p2[1])
            B = (p2[0] - p1[0])
            C = (p1[0]*p2[1] - p2[0]*p1[1])
            return A, B, -C

        def intersection(L1, L2):
            D  = L1[0] * L2[1] - L1[1] * L2[0]
            Dx = L1[2] * L2[1] - L1[1] * L2[2]
            Dy = L1[0] * L2[2] - L1[2] * L2[0]

            x = Dx / D
            y = Dy / D
            return x,y

        L1 = line([point1[0],point1[1]], [point2[0],point2[1]])
        L2 = line([point3[0],point3[1]], [point4[0],point4[1]])
        R = intersection(L1, L2)
        
        return R

    idx = np.argwhere(np.diff(np.sign(y1 - y2)) != 0)
    xc, yc = intercept((x[idx], y1[idx]),((x[idx+1], y1[idx+1])), ((x[idx], y2[idx])), ((x[idx+1], y2[idx+1])))
    return xc,yc

def line_simple(p1, p2):
    A = (p1[1] - p2[1])
    B = (p2[0] - p1[0])
    C = (p1[0]*p2[1] - p2[0]*p1[1])
    return A, B, -C
def intersection_simple(L1, L2):
    D  = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]

    x = Dx / D
    y = Dy / D
    return x,y

In [75]:
t0_interpol = np.zeros(len(file_path))
for to_plot in range(len(file_path)):
    H = diff(y_win[to_plot])/window_samp
    H_max = argmax(H)+5

    x  = array(y_win[to_plot][:H_max])
    y1 = H[:H_max]
    nv = len(y1)
    log_temps = x # linspace(-3.5,3,nv)
    coolpow_lowres = []
    for T in x: # 10**log_temps:
        v_thermique = sqrt(2*kb*T/m_Ca)
        dv = 10*v_thermique/nv
        coolpow_lowres.append( sum(cool_power(-dv*nv/2, dv*nv/2, nv, T, -Gam/2, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF ) 
    y2 = array(coolpow_lowres)[:H_max]
    try:
        idx = np.argwhere(np.diff(np.sign(y1 - y2)) != 0)
        # new method!
        xc, yc = interpolated_intercept(x,y1,y2)

        y1 = array(y_win[to_plot])
        y2 = xc[0]*np.ones(len(x_win[to_plot]))
        idx = np.argwhere(np.diff(np.sign(y1 - y2)) != 0)
        t0_interpol[to_plot], yct = interpolated_intercept(array(x_win[to_plot]), y1, y2)
        print('>',to_plot,t0_interpol[to_plot]*1e3,yct[0][0])
    except:
        print('>',to_plot,'skippity skippito')

> 0 skippity skippito
> 1 185.75732602834665 0.049417551179301605
> 2 65.85459192570904 0.03915425811437057
> 3 27.560584702860773 0.02885007874564071
> 4 0.3518982815806944 0.007811921645467113
> 5 3.113852253213327 0.022204161559127595
> 6 1.3811162665711203 0.01629635393188602
> 7 skippity skippito
> 8 0.25241372369417864 0.00894851012160676
> 9 75.05082433670465 0.04825804166472155
> 10 1.119001551545776 0.8658753375792123
> 11 198.8567838119323 0.05479163006731187
> 12 75.05082433670465 0.04825804166472155
> 13 28.90013171196691 0.037966684114856944
> 14 12.153010273642991 0.033663232290704094
> 15 0.9529801179011894 0.028993618270131204
> 16 1.8533186067497232 0.021019726421309375
> 17 skippity skippito
> 18 0.4607496990086017 0.011871782157389455
> 19 0.1598624128946476 0.010472624123673483
> 20 78.69061960417085 0.05296276666990369
> 21 skippity skippito
> 22 14.759642180017657 0.03636308441999227
> 23 6.553903024777313 0.03188270067680421
> 24 0.856837766382852 0.0278368920555

In [99]:
to_plot = 1
H = diff(y_win[to_plot])/window_samp
H_max = argmax(H)+5

x  = array(y_win[to_plot][:H_max])
y1 = H[:H_max]
nv = len(y1)
log_temps = x # linspace(-3.5,3,nv)
coolpow_lowres = []
for T in x: # 10**log_temps:
    v_thermique = sqrt(2*kb*T/m_Ca)
    dv = 10*v_thermique/nv
    coolpow_lowres.append( sum(cool_power(-dv*nv/2, dv*nv/2, nv, T, -Gam/2, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF ) 
y2 = array(coolpow_lowres)[:H_max]
idx = np.argwhere(np.diff(np.sign(y1 - y2)) != 0)
# new method!
xc, yc = interpolated_intercept(x,y1,y2)

figure('interpol',clear=True)
plt.plot(x, y1, marker='o', mec='none', ms=4, lw=1, label='y1')
plt.plot(x, y2, marker='o', mec='none', ms=4, lw=1, label='y2')
# plt.loglog(x[10], y1[10], marker='P', mec='none', ms=15, lw=1, label='y1')
plt.plot(x[idx], y1[idx], 'ms', ms=7, label='Nearest data-point method')
plt.plot(xc, yc, marker='o', ms=5,mew=1.5,color='none', mec='c', label='Nearest data-point, with linear interpolation')
xlim(-0.02,0.2)
ylim(-0.0001,0.002)
plt.legend(frameon=False, fontsize=10, numpoints=1, loc='lower left')
plt.savefig('curve crossing.png', dpi=200)
plt.grid()
plt.tight_layout()

In [77]:
t0_interpol = t0_interpol - fit_above_thresh

In [163]:
t0_interpol_rework = [t0_interpol[1:9],
              t0_interpol[12:20], # 7:15
              t0_interpol[20:]]

C = []
D = []
x_forfit = [linspace(0.555,0.71,200),linspace(0.555,0.71,200),linspace(0.535,0.71,200)]
figname = 'fit_parameter_t0_interpol_log'
fig = figure(figname,clear=True)
fig.set_size_inches(11.7, 8.3)
ax1 = subplot(111)
for k in range(1):
    ax1.loglog(q_rework[k],t0_interpol_rework[k]*1e3,
                 color=cm((k)/10),ls='',marker='v',ms=17,mew=0,label=N_rework[k])
    xx = hstack((q_rework[k][:3],q_rework[k][4:]))
    yy = hstack((delta_T_explosion_rework[k][:3],delta_T_explosion_rework[k][4:]))
    popt, pcov = curve_fit(func_lin, log10(array(xx)),log10(yy),maxfev= 5000)
    print(k,popt)
    C.append(exp(popt[1]))
    D.append(popt[0])
    ax1.loglog(x_forfit[k],func_pow(x_forfit[k],popt[0],popt[1])*1e3,
                 ls='--',lw=2,color=cm(k/10))
    
for k in range(1,2):
    ax1.loglog(q_rework[k],t0_interpol_rework[k]*1e3,
                 color=cm((k)/10),ls='',marker='D',ms=13,mew=0,label=N_rework[k])
    xx = hstack((q_rework[k][:3],q_rework[k][4:]))
    yy = hstack((delta_T_explosion_rework[k][:3],delta_T_explosion_rework[k][4:]))
    popt, pcov = curve_fit(func_lin, log10(array(xx)),log10(yy),maxfev= 5000)
    print(k,popt)
    C.append(exp(popt[1]))
    D.append(popt[0])
    ax1.loglog(x_forfit[k],func_pow(x_forfit[k],popt[0],popt[1])*1e3,
                 ls='--',lw=2,color=cm(k/10))
    
for k in range(2,3):
    ax1.loglog(q_rework[k][:9],t0_interpol_rework[k][:9]*1e3,
                 color=cm((k)/10),ls='',marker='P',ms=17,mew=0,label=N_rework[k])
    xx = hstack((q_rework[k][:4],q_rework[k][5:9]))
    yy = hstack((delta_T_explosion_rework[k][:4],delta_T_explosion_rework[k][5:9]))
    popt, pcov = curve_fit(func_lin, log10(array(xx)),log10(yy),maxfev= 5000)
    print(k,popt)
    C.append(exp(popt[1]))
    D.append(popt[0])
    ax1.loglog(x_forfit[k],func_pow(x_forfit[k],popt[0],popt[1])*1e3,
                 ls='--',lw=2,color=cm(k/10))
    
ax1.set_xlabel('$q_{x}$ (log scale)')
ax1.set_ylabel('$\Delta t$ (ms)')
ax1.grid()
    
# k = 2
# ax1.loglog(q_rework[2][:9],delta_T_explosion_rework[2][:9]*1e3,
#                  color=cm(2/10),ls='',marker='P',ms=17,mew=0,label=N_rework[2])
# popt, pcov = curve_fit(func_lin, log10(array(q_rework[2][:9])),
#                        log10(delta_T_explosion_rework[2][:9]),
#                        maxfev= 5000)
# print(k,popt)
# ax1.loglog(x_forfit[2],func_pow(x_forfit[2],popt[0],popt[1])*1e3,
#                  ls='--',lw=2,color=cm(2/10))
ax1.set_xticks([0.54,0.56,0.58,0.6,0.62,0.64,0.66,0.68,0.7])
# ax1.set_xticklabels(rotation=45, ha='right')
ax1.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax1.grid(visible=True, which='major', color='0.2', linestyle='-')
ax1.grid(visible=True, which='minor', color='0.7', linestyle='--')
# ax1.legend(title='N',fontsize=18)
tight_layout()
plt.savefig(figname+'.eps', dpi=300)



0 [-20.30085492  -7.01399697]
1 [-18.38802173  -6.58773946]
2 [-16.68375342  -6.09384478]


In [80]:
popt_rework

/usr/lib/python3/dist-packages/numpy/core/fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


(3,)

In [83]:
shape(popt_rework[0][:])

(8, 5)

In [108]:
A = array(popt_rework[0])[:,1]
B = array(popt_rework[0])[:,2]
C = array(popt_rework[0])[:,3]
kappa = array(popt_rework[0])[:,4]

figname = 'coucou_le_plot'
fig = figure(figname,clear=True)
fig.set_size_inches(11.7, 8.3)
ax1 = subplot(111)

ax1.plot(q_rework[0],exp(B),
             color=cm((k)/10),ls=':',marker='P',ms=17,mew=0)
ax1.set_xlabel('$q_{x}$')
ax1.set_ylabel('$e^B$')
ax1.grid()

# ax1.legend(title='N',fontsize=18)
tight_layout()

figname = 'coucou_le_plot_2'
fig = figure(figname,clear=True)
fig.set_size_inches(11.7, 8.3)
ax1 = subplot(111)

ax1.semilogy(q_rework[0],A*exp(B)*C**(-1/kappa)*tau_RF,
             color=cm((k)/10),ls=':',marker='P',ms=17,mew=0)
ax1.set_xlabel('$q_{x}$')
ax1.set_ylabel(r'$H(t_0)\; (K/\tau_{RF})$')
ax1.grid()

# ax1.legend(title='N',fontsize=18)
tight_layout()